In [ ]:
# @title
# USE_SAME_FEW_SHOTS=True

In [ ]:
# number_of_shots = 7

In [ ]:
nbit = 4
llamasize = '7B'
# llamasize = '13B'
temp = 0.1
# outdir = 'data/output/2023.10.20-2_LLama2_langchain/'
# outdir = 'data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/'
# outfilename = outdir+f'sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
# outfilename

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"

# INSTALLAZIONE E IMPORT LIBRERIE

In [ ]:
# @title
import pickle

In [ ]:
# @title
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title
import glob

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_2023.12.11.pickle"

experiment_name = "2024.02.15-00_LLama2_7B-chat-hf_4-bit_float-16_zero-shot"

output_data_path = base_path + "/../Llama-2/data/output/" + experiment_name
final_excel_filename = output_data_path+"/"+experiment_name+".xlsx"
classification_report_filename = output_data_path+f'/'+experiment_name+'_classification_report.xlsx'


drive.mount(mount_point, force_remount=True)

In [ ]:
# @title
with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [ ]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off

In [ ]:
# @title
# Installazione "permanente"
!pip install -q sentencepiece --progress-bar off && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [ ]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

In [ ]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [ ]:
# @title
torch.cuda.empty_cache()

# LOGIN DA HUGGING FACE

In [ ]:
# @title
# import os
# from IPython.display import clear_output
# os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
# clear_output()

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
# @title
!huggingface-cli login --token $huggingface_cli_token

# DOWNLOAD E CARICAMENTO DEL MODELLO

In [ ]:
# @title
# pip install --upgrade bitsandbytes

In [ ]:
# @title
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# !nvidia-smi

In [ ]:
# @title
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
model.eval()

In [ ]:
# @title
torch.cuda.empty_cache()

## TEST

In [ ]:
# @title
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
# data = pd.read_excel("data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# data = pd.read_excel("../data/input/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
# data

In [ ]:
# @title
data.shape

In [ ]:
# @title
# data.groupby('sdg').count()['text']

In [ ]:
# @title
data_sel = data

In [ ]:
# @title
# import pandas as pd
# data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(2)
# data_sel_4_few_shot_examples

In [ ]:
# @title
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [ ]:
# @title
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(7)

# data_sel_4_few_shot_examples

In [ ]:
# @title
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [ ]:
# @title
# template = """[INST]
# <<SYS>>
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:
# <</SYS>>

# '''
# {text}
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [ ]:
# @title
# prompt = PromptTemplate(
#     input_variables=["text"],
#     template=template,
# )

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# top7_train_data_sim_to_each_test_data

In [ ]:
# data_sel = data #.iloc[649:]
data_sel = data #.iloc[:60] # RIFACCIO LA CLASSIFICAZIONE DEI TESTI DELLA CLASSE SDG0
# data_sel = data[data.text_id==3844]

In [ ]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

#cnt = 0
# cnt = 650

results = []

for index, row in data_sel.iterrows():

  cnt = index

  print("\n\n##################################################")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  #few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  #few_shots = ""

  # for train_data_id in few_shots_list:

  #   # print("##################################################")
  #   # print(train_data_id)

  #   train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

  #   # print(train_row)

  #   # print(train_row.text.values[0])
  #   # print(train_row.sdg.values[0])

  #   input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
  #   output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

  #   few_shots += input_text
  #   few_shots += output_text

  # print(few_shots)

# RIMOSSO PER FARE ZERO SHOT!!!
# The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:
# """+few_shots+"""
  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  try:
    prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
    )
    result = llm(prompt.format(text=text))
  except Exception as e:
    result = str(e)
    print(cnt, text_id, sdg_true, result)

  result_extended = (cnt, text_id, sdg_true, result)

  print(result_extended)

  with open(output_data_path+f'/sdg_classification_Llama2_13B_zero-shot_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  #cnt+=1

  # if cnt==5:
  #   break

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result']).sort_values(['cnt'])
results_processed.head(30)

In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(60)

In [ ]:
data_sel = data

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final

In [ ]:
final.to_excel(final_excel_filename)

In [ ]:
final = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
classification_report_filename

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [ ]:
test = pd.read_excel(classification_report_filename)
test

In [ ]:
results_processed_merged = final
results_processed_merged = results_processed_merged.rename(columns={"label":"predicted_label_clean"})

In [ ]:
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: "0" if int(x)>16 else x)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
# report_df.to_excel(classification_report_filename)

In [ ]:
report_df

In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))